In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

2023-09-23 23:41:59.537115: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-23 23:41:59.586779: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 23:41:59.658967: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 23:41:59.659661: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 23:42:00.395647: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
data = pd.read_csv("data.csv")
data.head()

,Unnamed: 0.1,Unnamed: 0,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
0,0,2010-01-04,0.9133,1.4419,1.7200,1.0377,6.8273,5.1597,7.7555,46.27,92.55,3.396
1,1,2010-01-05,0.9143,1.4402,1.7296,1.0371,6.8258,5.1668,7.7564,46.13,91.48,3.385
2,2,2010-01-06,0.9189,1.4404,1.7292,1.0333,6.8272,5.1638,7.7546,45.72,92.53,3.379
3,3,2010-01-07,0.9168,1.4314,1.7409,1.0351,6.8280,5.1981,7.7539,45.67,93.31,3.368
4,4,2010-01-08,0.9218,1.4357,1.7342,1.0345,6.8274,5.1827,7.7553,45.50,92.70,3.375


In [3]:
def preprocess(data):
    data.drop("Unnamed: 0.1", axis = 1, inplace = True)
    data.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
    data["Date"] = pd.to_datetime(data["Date"])
    data.set_index("Date", inplace=True)
    data.replace(0, np.nan, inplace=True)
    display(data)
    print("Filling missing Values: ")
    display(data.interpolate(method='linear', limit_direction='forward'))
    data.interpolate(method='linear', limit_direction='forward', inplace = True)
    return data

In [4]:
data = preprocess(data)

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Date,,,,,,,,,,
2010-01-04,0.9133,1.4419,1.7200,1.0377,6.8273,5.1597,7.7555,46.27,92.55,3.3960
2010-01-05,0.9143,1.4402,1.7296,1.0371,6.8258,5.1668,7.7564,46.13,91.48,3.3850
2010-01-06,0.9189,1.4404,1.7292,1.0333,6.8272,5.1638,7.7546,45.72,92.53,3.3790
2010-01-07,0.9168,1.4314,1.7409,1.0351,6.8280,5.1981,7.7539,45.67,93.31,3.3680
2010-01-08,0.9218,1.4357,1.7342,1.0345,6.8274,5.1827,7.7553,45.50,92.70,3.3750
...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.6978,1.1174,4.0507,1.3073,6.9954,6.6829,7.7874,71.45,109.47,4.1260
2019-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filling missing Values: 


,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Date,,,,,,,,,,
2010-01-04,0.913300,1.441900,1.720000,1.0377,6.8273,5.1597,7.755500,46.27,92.550000,3.3960
2010-01-05,0.914300,1.440200,1.729600,1.0371,6.8258,5.1668,7.756400,46.13,91.480000,3.3850
2010-01-06,0.918900,1.440400,1.729200,1.0333,6.8272,5.1638,7.754600,45.72,92.530000,3.3790
2010-01-07,0.916800,1.431400,1.740900,1.0351,6.8280,5.1981,7.753900,45.67,93.310000,3.3680
2010-01-08,0.921800,1.435700,1.734200,1.0345,6.8274,5.1827,7.755300,45.50,92.700000,3.3750
...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.697800,1.117400,4.050700,1.3073,6.9954,6.6829,7.787400,71.45,109.470000,4.1260
2019-12-28,0.698667,1.118833,4.038867,1.3068,6.9924,6.6749,7.786833,71.40,109.263333,4.1191
2019-12-29,0.699533,1.120267,4.027033,1.3063,6.9894,6.6669,7.786267,71.35,109.056667,4.1122


In [5]:
LOOK_BACK = 30
PREDICT_DAY = 1
SPLIT_RATIO = 0.8

In [ ]:
def Recurrent_reshape(data, lookback = LOOK_BACK):
    return data.reshape(-1,1,lookback)

In [6]:
def Create_Data(data, lookback = LOOK_BACK, pred_len = PREDICT_DAY, split_ratio = SPLIT_RATIO, model = "FNN"):
    if lookback<2:
        print("ERROR: Lookback too small")
        return -1

# declarations

    x = {}
    y = {}
    xtr = {}
    xt = {}
    ytr = {}
    yt = {}
    scalers = {}

# Creating stepped data

    for i in data.columns:
        xtemp = pd.DataFrame(data[i])
        for j in range(1,lookback+1):
            xtemp[i+str(j)] = data[i].shift(-1*j)
        x[i] = xtemp.dropna()

# Splitting data into x and y
        
    for i in x.keys():
        y[i] = pd.DataFrame(x[i].iloc[:,-pred_len])
        x[i] = x[i].iloc[:,:-pred_len]
        
# Normalizing x and y values
        
    for i in x.keys():
        scalers[i+"_x"] = MinMaxScaler(feature_range=(0,1))
        x[i] = scalers[i+"_x"].fit_transform(x[i])
        scalers[i+"_y"] = MinMaxScaler(feature_range=(0,1))
        y[i] = scalers[i+"_y"].fit_transform(y[i])

# setting train and test sizes
        
    tr_len = int(split_ratio*y["India"].shape[0])
    t_len = y["India"].shape[0]-tr_len
    
# creating training and testing data
    
    for i in x.keys():
        xtr[i] = x[i][:tr_len]
        ytr[i] = y[i][:tr_len]
        xt[i] = x[i][-t_len:]
        yt[i] = y[i][-t_len:]
        
# returning pertinent data
    
    return x,y,xtr,xt,ytr,yt,scalers

In [7]:
x,y,xtr,xt,ytr,yt,scalers = Create_Data(data, model="RNN")

In [ ]:
xtr["India"].shape

In [8]:
def Create_model(x,y, lookback = LOOK_BACK, Pred_size = PREDICT_DAY):
    models = {}
    for i in x.keys():
        models[i] = Sequential()
        models[i].add(SimpleRNN(32,input_shape=(LOOK_BACK,1,),return_sequences=True,activation="relu"))
        models[i].add(SimpleRNN(64,return_sequences=True,activation="relu"))
        models[i].add(Dropout(0.2))
        models[i].add(SimpleRNN(128,return_sequences=True,activation="relu"))
        models[i].add(Dropout(0.2))
        models[i].add(SimpleRNN(64,return_sequences=True,activation="relu"))
        models[i].add(Dropout(0.2))
        models[i].add(SimpleRNN(16,activation="relu"))
        models[i].add(Dense(Pred_size))
        models[i].compile(loss="mean_squared_error", optimizer="adam")
        print(i)
        display(models[i].summary())
    return models

In [9]:
m = Create_model(x,y)

Australia
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 30, 32)            1088      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 30, 64)            6208      
                                                                 
 dropout (Dropout)           (None, 30, 64)            0         
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 30, 128)           24704     
                                                                 
 dropout_1 (Dropout)         (None, 30, 128)           0         
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 30, 64)            12352     
                                                                 
 dropout_2 (Dropout)         (None, 30, 64)   

None

Europe
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_5 (SimpleRNN)    (None, 30, 32)            1088      
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 30, 64)            6208      
                                                                 
 dropout_3 (Dropout)         (None, 30, 64)            0         
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 30, 128)           24704     
                                                                 
 dropout_4 (Dropout)         (None, 30, 128)           0         
                                                                 
 simple_rnn_8 (SimpleRNN)    (None, 30, 64)            12352     
                                                                 
 dropout_5 (Dropout)         (None, 30, 64)    

None

Brazil
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_10 (SimpleRNN)   (None, 30, 32)            1088      
                                                                 
 simple_rnn_11 (SimpleRNN)   (None, 30, 64)            6208      
                                                                 
 dropout_6 (Dropout)         (None, 30, 64)            0         
                                                                 
 simple_rnn_12 (SimpleRNN)   (None, 30, 128)           24704     
                                                                 
 dropout_7 (Dropout)         (None, 30, 128)           0         
                                                                 
 simple_rnn_13 (SimpleRNN)   (None, 30, 64)            12352     
                                                                 
 dropout_8 (Dropout)         (None, 30, 64)    

...

None

Japan
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_40 (SimpleRNN)   (None, 30, 32)            1088      
                                                                 
 simple_rnn_41 (SimpleRNN)   (None, 30, 64)            6208      
                                                                 
 dropout_24 (Dropout)        (None, 30, 64)            0         
                                                                 
 simple_rnn_42 (SimpleRNN)   (None, 30, 128)           24704     
                                                                 
 dropout_25 (Dropout)        (None, 30, 128)           0         
                                                                 
 simple_rnn_43 (SimpleRNN)   (None, 30, 64)            12352     
                                                                 
 dropout_26 (Dropout)        (None, 30, 64)     

None

Malaysia
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_45 (SimpleRNN)   (None, 30, 32)            1088      
                                                                 
 simple_rnn_46 (SimpleRNN)   (None, 30, 64)            6208      
                                                                 
 dropout_27 (Dropout)        (None, 30, 64)            0         
                                                                 
 simple_rnn_47 (SimpleRNN)   (None, 30, 128)           24704     
                                                                 
 dropout_28 (Dropout)        (None, 30, 128)           0         
                                                                 
 simple_rnn_48 (SimpleRNN)   (None, 30, 64)            12352     
                                                                 
 dropout_29 (Dropout)        (None, 30, 64)  

None

In [10]:
def Execute_model(model,xtr,ytr,xt,yt, scaler):
    MAPE = {}
    MAE = {}
    MSE = {}
    for i in model.keys():
        print(i)
# Training the model
        
        model[i].fit(xtr[i], ytr[i], epochs=10, batch_size=1, verbose=1)
        
# collecting predicted and actual values
        
        temp = model[i].predict(xt[i])
        pred = scaler[i+"_y"].inverse_transform(temp)
        act = scaler[i+"_y"].inverse_transform(yt[i])
        
# calculating Mean Square Error, Mean Absolute Error, and Mean Absolute Error

        MSE[i] = mean_squared_error(act,pred)
        MAE[i] = mean_absolute_error(act,pred)
        MAPE[i] = mean_absolute_percentage_error(act, pred)
        
# Tabulating Data

    results = pd.DataFrame([MSE,MAE,MAPE])
    results["Metric"] = ["MSE","MAE","MAPE"]
    results.set_index("Metric",inplace=True)
    
    return results


In [11]:
result = Execute_model(m,xtr,ytr,xt,yt,scalers)

Australia
Epoch 1/10


2023-09-23 23:42:22.288726: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential/simple_rnn_3/while/body/_240/sequential/simple_rnn_3/while/simple_rnn_cell_3/Relu}}
	.  Registered:  <no registered kernels>

2023-09-23 23:42:22.290372: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential/simple_rnn_2/while/body/_157/sequential/simple_rnn_2/while/simple_rnn_cell_2/Relu}}
	.  Registered:  <no registered kernels>

2023-09-23 23:42:22.291595: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential/simple_rnn_1/while/body/_74/sequential/simple_rnn_1/while/simple_rnn_cell_1/Relu}}
	.  Registered:  <no registered kernels>

2023-09-23 23:42:22.292615: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with no

2894/2894 [==============================] - 37s 12ms/step - loss: 0.0061
Epoch 2/10
2894/2894 [==============================] - 42s 15ms/step - loss: 0.0024
Epoch 3/10
2894/2894 [==============================] - 49s 17ms/step - loss: 0.0019
Epoch 4/10
2894/2894 [==============================] - 48s 17ms/step - loss: 0.0015
Epoch 5/10
2894/2894 [==============================] - 47s 16ms/step - loss: 0.0015
Epoch 6/10
2894/2894 [==============================] - 47s 16ms/step - loss: 0.0013
Epoch 7/10
2894/2894 [==============================] - 48s 17ms/step - loss: 0.0013
Epoch 8/10
2894/2894 [==============================] - 53s 18ms/step - loss: 0.0011
Epoch 9/10
2894/2894 [==============================] - 46s 16ms/step - loss: 0.0011
Epoch 10/10
23/23 [==============================] - 0s 4ms/step
Europe
Epoch 1/10


2023-09-23 23:50:03.746326: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential_1/simple_rnn_8/while/body/_240/sequential_1/simple_rnn_8/while/simple_rnn_cell_8/Relu}}
	.  Registered:  <no registered kernels>

2023-09-23 23:50:03.747994: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential_1/simple_rnn_7/while/body/_157/sequential_1/simple_rnn_7/while/simple_rnn_cell_7/Relu}}
	.  Registered:  <no registered kernels>

2023-09-23 23:50:03.749110: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential_1/simple_rnn_6/while/body/_74/sequential_1/simple_rnn_6/while/simple_rnn_cell_6/Relu}}
	.  Registered:  <no registered kernels>

2023-09-23 23:50:03.750081: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compat

2894/2894 [==============================] - 46s 15ms/step - loss: 0.0050
Epoch 2/10
2894/2894 [==============================] - 47s 16ms/step - loss: 0.0020
Epoch 3/10
2894/2894 [==============================] - 49s 17ms/step - loss: 0.0017
Epoch 4/10
2894/2894 [==============================] - 47s 16ms/step - loss: 0.0014
Epoch 5/10
2894/2894 [==============================] - 48s 16ms/step - loss: 0.0013
Epoch 6/10
2894/2894 [==============================] - 48s 16ms/step - loss: 0.0012
Epoch 7/10
2894/2894 [==============================] - 54s 19ms/step - loss: 0.0011
Epoch 8/10
2894/2894 [==============================] - 49s 17ms/step - loss: 0.0011
Epoch 9/10
2894/2894 [==============================] - 49s 17ms/step - loss: 9.9434e-04
Epoch 10/10
23/23 [==============================] - 0s 5ms/step
Brazil
Epoch 1/10


...

2894/2894 [==============================] - 53s 17ms/step - loss: 0.0066
Epoch 2/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0023
Epoch 3/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0020
Epoch 4/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0016
Epoch 5/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0014
Epoch 6/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0012
Epoch 7/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0013
Epoch 8/10
2894/2894 [==============================] - 50s 17ms/step - loss: 9.9314e-04
Epoch 9/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0010
Epoch 10/10
23/23 [==============================] - 1s 5ms/step
Malaysia
Epoch 1/10


2023-09-24 00:56:42.947856: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential_9/simple_rnn_48/while/body/_240/sequential_9/simple_rnn_48/while/simple_rnn_cell_48/Relu}}
	.  Registered:  <no registered kernels>

2023-09-24 00:56:42.949565: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential_9/simple_rnn_47/while/body/_157/sequential_9/simple_rnn_47/while/simple_rnn_cell_47/Relu}}
	.  Registered:  <no registered kernels>

2023-09-24 00:56:42.950735: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devices compatible with node {{node sequential_9/simple_rnn_46/while/body/_74/sequential_9/simple_rnn_46/while/simple_rnn_cell_46/Relu}}
	.  Registered:  <no registered kernels>

2023-09-24 00:56:42.951926: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered '' OpKernel for CPU devic

   1/2894 [..............................] - ETA: 2:04:34 - loss: 0.0043

2023-09-24 00:56:43.150287: E tensorflow/core/grappler/optimizers/tfg_optimizer_hook.cc:134] tfg_optimizer{any(tfg-consolidate-attrs,tfg-functional-to-region,tfg.func(tfg-cf-sink),tfg-region-to-functional{force-control-capture=true},tfg-lift-legacy-call,symbol-privatize{},symbol-dce,tfg-prepare-attrs-export)} failed: INVALID_ARGUMENT: Node sequential_9/simple_rnn_45/while/body/_1/sequential_9/simple_rnn_45/while/simple_rnn_cell_45/Relu has an empty op name
	when importing GraphDef to MLIR module in GrapplerHook
2023-09-24 00:56:43.153114: E tensorflow/core/grappler/optimizers/tfg_optimizer_hook.cc:134] tfg_optimizer{any(tfg-consolidate-attrs,tfg-functional-to-region,tfg.func(tfg-cf-sink),tfg-region-to-functional{force-control-capture=true},tfg-lift-legacy-call,symbol-privatize{},symbol-dce,tfg-prepare-attrs-export)} failed: INVALID_ARGUMENT: Node sequential_9/simple_rnn_45/while/body/_1/sequential_9/simple_rnn_45/while/simple_rnn_cell_45/Relu has an empty op name
	when importing GraphD

2894/2894 [==============================] - 53s 17ms/step - loss: 0.0056
Epoch 2/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0025
Epoch 3/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0016
Epoch 4/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0014
Epoch 5/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0014
Epoch 6/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0012
Epoch 7/10
2894/2894 [==============================] - 51s 17ms/step - loss: 0.0011
Epoch 8/10
2894/2894 [==============================] - 50s 17ms/step - loss: 0.0010
Epoch 9/10
2894/2894 [==============================] - 50s 17ms/step - loss: 9.1448e-04
Epoch 10/10
23/23 [==============================] - 1s 6ms/step


In [12]:
result

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Metric,,,,,,,,,,
MSE,0.000089,0.000032,0.004377,0.000172,0.009904,0.006569,0.000032,0.182733,2.500005,0.003151
MAE,0.008171,0.004361,0.049615,0.011813,0.081518,0.069576,0.004955,0.330824,1.490829,0.048640
MAPE,0.011536,0.003795,0.012656,0.008951,0.011786,0.010661,0.000632,0.004706,0.013522,0.011756
